In [1]:
import osmnx as ox
import networkx as nx
import random

# 1. Load city graph (example: Algiers)
place = "Algiers, Algeria"
G = ox.graph_from_place(place, network_type="drive")

# Ensure graph is undirected (better for routing)
G = G.to_undirected()

# 2. Generate drivers and passengers
num_drivers = 3
num_passengers = 6

# Pick random nodes for drivers and passengers
nodes = list(G.nodes)

drivers = [{"id": f"D{i+1}", "node": random.choice(nodes), "capacity": 3} for i in range(num_drivers)]
passengers = [{"id": f"P{j+1}", 
               "origin": random.choice(nodes), 
               "dest": random.choice(nodes)} for j in range(num_passengers)]

print("Drivers:", drivers)
print("Passengers:", passengers)

# 3. Compute shortest paths & assign passengers to nearest driver
assignments = {}
for passenger in passengers:
    best_driver = None
    best_distance = float("inf")
    
    for driver in drivers:
        try:
            # compute shortest path length between driver and passenger origin
            dist = nx.shortest_path_length(G, driver["node"], passenger["origin"], weight="length")
        except nx.NetworkXNoPath:
            continue
        
        if dist < best_distance and driver["capacity"] > 0:
            best_distance = dist
            best_driver = driver
    
    if best_driver:
        # Assign passenger to this driver
        assignments.setdefault(best_driver["id"], []).append(passenger["id"])
        best_driver["capacity"] -= 1

print("\nAssignments (Greedy Matching):")
for d, p_list in assignments.items():
    print(f"Driver {d} -> Passengers {p_list}")


Drivers: [{'id': 'D1', 'node': 10559702627, 'capacity': 3}, {'id': 'D2', 'node': 312773703, 'capacity': 3}, {'id': 'D3', 'node': 284134693, 'capacity': 3}]
Passengers: [{'id': 'P1', 'origin': 3923238261, 'dest': 2534632804}, {'id': 'P2', 'origin': 381441785, 'dest': 9351932491}, {'id': 'P3', 'origin': 12448348460, 'dest': 392967047}, {'id': 'P4', 'origin': 347388177, 'dest': 530658399}, {'id': 'P5', 'origin': 535861375, 'dest': 10762205497}, {'id': 'P6', 'origin': 326822242, 'dest': 11652526848}]

Assignments (Greedy Matching):
Driver D2 -> Passengers ['P1', 'P2', 'P3']
Driver D3 -> Passengers ['P4', 'P5', 'P6']


Added MAX_PICKUP_DISTANCE: passenger must be close enough.

Added MAX_DETOUR_FACTOR: prevents long detours.

Driver capacity updated after each assignment.

Driver moves to passenger destination after dropoff.

In [2]:
import osmnx as ox
import networkx as nx
import random

# 1. Load city graph
place = "Algiers, Algeria"
G = ox.graph_from_place(place, network_type="drive")
G = G.to_undirected()

# 2. Generate drivers and passengers
num_drivers = 3
num_passengers = 6
nodes = list(G.nodes)

drivers = [
    {"id": f"D{i+1}", "node": random.choice(nodes), "capacity": 3, "route": []}
    for i in range(num_drivers)
]
passengers = [
    {"id": f"P{j+1}", "origin": random.choice(nodes), "dest": random.choice(nodes)}
    for j in range(num_passengers)
]

# Constraints
MAX_PICKUP_DISTANCE = 1500  # meters (e.g. passenger must be <1.5 km away)
MAX_DETOUR_FACTOR = 1.4     # driver trip length should not exceed 1.4x original

# Helper: compute path length
def path_length(G, path):
    return sum(G[u][v][0].get("length", 0) for u, v in zip(path[:-1], path[1:]))

# Compute base routes (driver -> random "standby destination" if empty)
for driver in drivers:
    # initial route: taxi waits at its node (or could have fixed destination)
    driver["route"] = [driver["node"]]

assignments = {}

# 3. Assign passengers with constraints
for passenger in passengers:
    best_driver = None
    best_extra_dist = float("inf")

    for driver in drivers:
        if driver["capacity"] <= 0:
            continue

        try:
            # Distance driver → passenger
            pickup_dist = nx.shortest_path_length(G, driver["node"], passenger["origin"], weight="length")
        except nx.NetworkXNoPath:
            continue

        if pickup_dist > MAX_PICKUP_DISTANCE:
            continue  # too far, reject

        # Compute detour effect if assigned
        try:
            base_path = nx.shortest_path(G, driver["node"], passenger["dest"], weight="length")
            base_len = path_length(G, base_path)

            with_passenger = (
                nx.shortest_path_length(G, driver["node"], passenger["origin"], weight="length")
                + nx.shortest_path_length(G, passenger["origin"], passenger["dest"], weight="length")
            )
        except nx.NetworkXNoPath:
            continue

        detour_ratio = with_passenger / base_len if base_len > 0 else 1

        if detour_ratio > MAX_DETOUR_FACTOR:
            continue  # detour too large

        # Choose driver with minimal extra distance
        if with_passenger < best_extra_dist:
            best_extra_dist = with_passenger
            best_driver = driver

    if best_driver:
        assignments.setdefault(best_driver["id"], []).append(passenger["id"])
        best_driver["capacity"] -= 1
        best_driver["node"] = passenger["dest"]  # update taxi position

# 4. Print results
print("Drivers:", drivers)
print("Passengers:", passengers)
print("\nAssignments with constraints:")
for d, p_list in assignments.items():
    print(f"Driver {d} -> Passengers {p_list}")


Drivers: [{'id': 'D1', 'node': 346871094, 'capacity': 3, 'route': [346871094]}, {'id': 'D2', 'node': 2705961543, 'capacity': 3, 'route': [2705961543]}, {'id': 'D3', 'node': 7689146577, 'capacity': 3, 'route': [7689146577]}]
Passengers: [{'id': 'P1', 'origin': 4051049610, 'dest': 535862168}, {'id': 'P2', 'origin': 4265949492, 'dest': 2932733509}, {'id': 'P3', 'origin': 326566180, 'dest': 2372856106}, {'id': 'P4', 'origin': 10803884670, 'dest': 6520316429}, {'id': 'P5', 'origin': 10904902842, 'dest': 319879639}, {'id': 'P6', 'origin': 10889999092, 'dest': 3821983153}]

Assignments with constraints:


In [4]:
import osmnx as ox
import networkx as nx
import random

# 1. Load city graph
place = "Algiers, Algeria"
G = ox.graph_from_place(place, network_type="drive")
G = G.to_undirected()

# 2. Initialize drivers
num_drivers = 3
nodes = list(G.nodes)

drivers = [
    {"id": f"D{i+1}", "node": random.choice(nodes), "capacity": 3, "onboard": [], "route": []}
    for i in range(num_drivers)
]

# Dynamic constraints
BASE_PICKUP_DISTANCE = 1500
BASE_DETOUR_FACTOR = 1.4
MAX_PICKUP_DISTANCE = 4000   # hard limit
MAX_DETOUR_FACTOR = 2.5      # hard limit

# Helper: path length
def path_length(G, path):
    return sum(G[u][v][0].get("length", 0) for u, v in zip(path[:-1], path[1:]))

# Dynamic simulation
time_steps = 10
all_passengers = []
assignments = {}

for t in range(time_steps):
    print(f"\n⏱️ Time step {t}")

    # New passengers appear randomly
    if random.random() < 0.5:  # 50% chance new passenger arrives
        new_passenger = {
            "id": f"P{len(all_passengers)+1}",
            "origin": random.choice(nodes),
            "dest": random.choice(nodes),
            "status": "waiting"
        }
        all_passengers.append(new_passenger)
        print(f"🆕 New passenger {new_passenger['id']} appeared")

    # Assign waiting passengers
    for passenger in [p for p in all_passengers if p["status"] == "waiting"]:
        best_driver = None
        best_extra_dist = float("inf")

        # Start with base constraints
        pickup_limit = BASE_PICKUP_DISTANCE
        detour_limit = BASE_DETOUR_FACTOR

        while best_driver is None and pickup_limit <= MAX_PICKUP_DISTANCE and detour_limit <= MAX_DETOUR_FACTOR:
            for driver in drivers:
                if driver["capacity"] <= 0:
                    continue

                try:
                    pickup_dist = nx.shortest_path_length(G, driver["node"], passenger["origin"], weight="length")
                except nx.NetworkXNoPath:
                    continue

                if pickup_dist > pickup_limit:
                    continue

                try:
                    base_path = nx.shortest_path(G, driver["node"], passenger["dest"], weight="length")
                    base_len = path_length(G, base_path)

                    with_passenger = (
                        nx.shortest_path_length(G, driver["node"], passenger["origin"], weight="length")
                        + nx.shortest_path_length(G, passenger["origin"], passenger["dest"], weight="length")
                    )
                except nx.NetworkXNoPath:
                    continue

                detour_ratio = with_passenger / base_len if base_len > 0 else 1
                if detour_ratio > detour_limit:
                    continue

                if with_passenger < best_extra_dist:
                    best_extra_dist = with_passenger
                    best_driver = driver

            # If no driver found, relax constraints
            if best_driver is None:
                pickup_limit += 500
                detour_limit += 0.2
                print(f"⚠️ Relaxing constraints: pickup ≤ {pickup_limit}m, detour ≤ {detour_limit:.1f}")

        if best_driver:
            assignments.setdefault(best_driver["id"], []).append(passenger["id"])
            best_driver["capacity"] -= 1
            best_driver["onboard"].append(passenger["id"])
            best_driver["node"] = passenger["dest"]  # driver moves
            passenger["status"] = "assigned"
            print(f"✅ Assigned {passenger['id']} to {best_driver['id']} with constraints: "
                  f"pickup ≤ {pickup_limit}m, detour ≤ {detour_limit:.1f}")
        else:
            print(f"❌ Passenger {passenger['id']} could not be assigned (even after relaxing).")

    # Drivers drop off onboard passengers (simulate ride completed)
    for driver in drivers:
        if driver["onboard"]:
            dropped = driver["onboard"].pop(0)
            driver["capacity"] += 1
            passenger = next(p for p in all_passengers if p["id"] == dropped)
            passenger["status"] = "dropped"
            print(f"🚖 Driver {driver['id']} dropped {dropped}")

# Final report
print("\n📊 Final Assignments:")
for d, p_list in assignments.items():
    print(f"{d} -> {p_list}")



⏱️ Time step 0

⏱️ Time step 1

⏱️ Time step 2

⏱️ Time step 3

⏱️ Time step 4
🆕 New passenger P1 appeared
⚠️ Relaxing constraints: pickup ≤ 2000m, detour ≤ 1.6
⚠️ Relaxing constraints: pickup ≤ 2500m, detour ≤ 1.8
⚠️ Relaxing constraints: pickup ≤ 3000m, detour ≤ 2.0
⚠️ Relaxing constraints: pickup ≤ 3500m, detour ≤ 2.2
⚠️ Relaxing constraints: pickup ≤ 4000m, detour ≤ 2.4
⚠️ Relaxing constraints: pickup ≤ 4500m, detour ≤ 2.6
❌ Passenger P1 could not be assigned (even after relaxing).

⏱️ Time step 5
🆕 New passenger P2 appeared
⚠️ Relaxing constraints: pickup ≤ 2000m, detour ≤ 1.6
⚠️ Relaxing constraints: pickup ≤ 2500m, detour ≤ 1.8
⚠️ Relaxing constraints: pickup ≤ 3000m, detour ≤ 2.0
⚠️ Relaxing constraints: pickup ≤ 3500m, detour ≤ 2.2
⚠️ Relaxing constraints: pickup ≤ 4000m, detour ≤ 2.4
⚠️ Relaxing constraints: pickup ≤ 4500m, detour ≤ 2.6
❌ Passenger P1 could not be assigned (even after relaxing).
⚠️ Relaxing constraints: pickup ≤ 2000m, detour ≤ 1.6
⚠️ Relaxing constraints: p